<a href="https://colab.research.google.com/github/MrSuperfluous/Automata/blob/main/SummerAnalCapstone_pw(included).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("dataset.csv")

# Combine date + time
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")

# Save selected columns for streaming
df_stream = df[[
    "Timestamp", "SystemCodeNumber", "Occupancy", "Capacity", "QueueLength",
    "TrafficConditionNearby", "IsSpecialDay", "VehicleType", "Latitude", "Longitude"
]]

# Encode features
df_stream["VehicleTypeWeight"] = df_stream["VehicleType"].map({
    "car": 1.0, "bike": 0.5, "truck": 1.5
}).fillna(1.0)

df_stream["TrafficLevel"] = df_stream["TrafficConditionNearby"].map({
    "low": 1, "medium": 2, "high": 3
}).fillna(2)

df_stream.to_csv("./parking/parking_stream.csv", index=False)


/tmp/ipython-input-1-3557626570.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stream["VehicleTypeWeight"] = df_stream["VehicleType"].map({
/tmp/ipython-input-1-3557626570.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stream["TrafficLevel"] = df_stream["TrafficConditionNearby"].map({


In [2]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [5]:
# ✅ Imports
import pandas as pd
import numpy as np
import pathway as pw
import panel as pn
import bokeh.plotting
from bokeh.models import HoverTool
from math import radians, sin, cos, asin, sqrt

pn.extension()

# ✅ Pathway schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: float
    Capacity: float
    QueueLength: float
    TrafficLevel: float
    IsSpecialDay: float
    VehicleTypeWeight: float
    Latitude: float
    Longitude: float

# ✅ Distance + Pricing
lot_coords = {}
current_prices = {}

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

@pw.udf
def compute_final_price(
    occupancy, capacity, queue_length, traffic_level,
    is_special_day, vehicle_type_weight,
    latitude, longitude, system_code_number
) -> float:
    lot_coords[system_code_number] = (latitude, longitude)
    occ_rate = occupancy / capacity if capacity > 0 else 0
    raw_demand = (
        1.0 * occ_rate +
        0.3 * queue_length -
        0.2 * traffic_level +
        0.6 * is_special_day +
        0.5 * vehicle_type_weight
    )

    # 🚀 More sensitive normalization
    norm = min(max(raw_demand / 6, 0), 1)
    base_price = 10 + 6 * (norm ** 2)  # 📈 more aggressive growth
    base_price = min(max(base_price, 5.0), 25.0)

    # 💡 Competitive logic
    lat1, lon1 = latitude, longitude
    nearby_prices = []
    for other_lot, other_price in current_prices.items():
        if other_lot == system_code_number or other_lot not in lot_coords:
            continue
        lat2, lon2 = lot_coords[other_lot]
        d = haversine(lat1, lon1, lat2, lon2)
        if d <= 1.0:
            nearby_prices.append(other_price)

    final_price = base_price
    if occ_rate >= 0.90 and any(p < base_price for p in nearby_prices):
        final_price = max(base_price - 0.5, 5.0)
    elif any(p > base_price for p in nearby_prices):
        final_price = min(base_price + 1.5, 25.0)

    current_prices[system_code_number] = final_price
    return float(final_price)

@pw.udf
def compute_reroute_flag(occupancy, capacity, final_price):
    occ_rate = occupancy / capacity if capacity > 0 else 0
    return occ_rate >= 0.95

# ✅ Stream input data
table = pw.io.csv.read(
    "./parking/",
    schema=ParkingSchema,
    mode="streaming"
)

# ✅ Apply pricing
with_price = table.select(
    *pw.this,
    FinalPrice=compute_final_price(
        pw.this.Occupancy, pw.this.Capacity, pw.this.QueueLength,
        pw.this.TrafficLevel, pw.this.IsSpecialDay,
        pw.this.VehicleTypeWeight, pw.this.Latitude,
        pw.this.Longitude, pw.this.SystemCodeNumber
    )
)

# ✅ Add reroute flag
result = with_price.select(
    *pw.this,
    RerouteFlag=compute_reroute_flag(
        pw.this.Occupancy, pw.this.Capacity, pw.this.FinalPrice
    )
)

# ✅ Add formatted timestamp for plotting
plotting_data = result.select(
    t=pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    price=pw.this.FinalPrice,
    occupancy=pw.this.Occupancy,
    capacity=pw.this.Capacity,
    reroute=pw.this.RerouteFlag,
    lot=pw.this.SystemCodeNumber
)

# ✅ Plotting functions
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=300, width=800,
        title="📊 Real-Time Price",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=5, color="red")
    fig.add_tools(HoverTool(tooltips=[("Time", "@t{%F %T}"), ("Price", "@price")], formatters={"@t": "datetime"}))
    return fig

def occupancy_plotter(source):
    fig = bokeh.plotting.figure(
        height=250, width=800,
        title="📉 Occupancy vs Capacity",
        x_axis_type="datetime"
    )
    fig.line("t", "occupancy", source=source, color="green", legend_label="Occupancy")
    fig.line("t", "capacity", source=source, color="blue", legend_label="Capacity")
    fig.scatter("t", "occupancy", source=source, size=4, color="green")
    fig.scatter("t", "capacity", source=source, size=4, color="blue")
    fig.legend.location = "top_left"
    return fig

def reroute_plotter(source):
    fig = bokeh.plotting.figure(
        height=150, width=800,
        title="🚦 Reroute Flag",
        x_axis_type="datetime"
    )
    fig.step("t", "reroute", source=source, line_width=3, color="red", mode="after")
    return fig

# ✅ All 14 lots from your image
lot_names = [
    "BHMBCCMKT01", "BHMBCCTHL01", "BHMEURBRD01", "BHMMBMMBX01", "BHMNCPHST01",
    "BHMNCPNST01", "Broad Street", "Others-CCCPS105a", "Others-CCCPS119a",
    "Others-CCCPS135a", "Others-CCCPS202", "Others-CCCPS8", "Others-CCCPS98",
    "Shopping"
]

# ✅ Build dashboard for each lot
def build_lot_dashboard(lot_name):
    lot_data = plotting_data.filter(plotting_data.lot == lot_name)

    return pn.Column(
        pn.pane.Markdown(f"## 🚗 Lot: `{lot_name}`"),
        lot_data.plot(price_plotter, sorting_col="t"),
        lot_data.plot(occupancy_plotter, sorting_col="t"),
        lot_data.plot(reroute_plotter, sorting_col="t"),
        pn.Spacer(height=40)
    )

# ✅ Final Panel dashboard
dashboard = pn.Column(
    pn.pane.Markdown("# 🔁 Real-Time Parking Dashboard for All 14 Lots"),
    *[build_lot_dashboard(lot) for lot in lot_names]
)

dashboard.servable()


Column
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [2] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [3] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [4] Spacer(height=40)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [2] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [3] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [4] Spacer(height=40)
    [3] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [2] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [3] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [4] Spacer(height=40)
    [4] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [2] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [3] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [4] Spacer(height=40)
    [5] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [2] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [3] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [4] Spacer(height=40)
    [6] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [2] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [3] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [4] Spacer(height=40)
    [7] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [2] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [3] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [4] Spacer(height=40)
    [8] Column
        [0] Markdown(str)
        [1] Column
            

In [ ]:
%%capture --no-display
pw.run()

Output()